<a href="https://colab.research.google.com/github/HongJaeKwon/kimhae_fire_detection/blob/master/hjk_fire_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# /gdrive/My Drive/ (폴더명)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from  sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input,BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, Iterator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.python.keras.losses import mean_absolute_error
warnings.filterwarnings(action='ignore') 

In [0]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def hjk_loss(y_true,y_pred):
    bce = 3*y_true * K.log(y_pred + K.epsilon())
    bce += (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    return -bce

def f1_loss(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = K.clip(f1,K.epsilon(),1-K.epsilon())
    return 1 - K.mean(f1)     

In [4]:
cnn_model=Sequential()
cnn_model.add(Dense(16*16,activation='relu',input_shape=(200,)))
cnn_model.add(Reshape((16,16,1)))
cnn_model.add(Conv2D(2, 3, activation='elu', strides=2, padding='same'))
cnn_model.add(Conv2D(4, 3, activation='elu', strides=2, padding='same'))
cnn_model.add(Conv2D(8, 3, activation='elu', strides=2, padding='same'))
cnn_model.add(Flatten())
cnn_model.add(Dense(1,activation='sigmoid'))
cnn_model.compile(loss=hjk_loss, metrics=['acc',f1_m, precision_m, recall_m],optimizer=Adam(0.001))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
all_DF= pd.read_csv('/gdrive/My Drive/김해화재/all_DF.csv',index_col=0)
# train :59199
# val : 6898
# test : 2957

In [35]:
all_DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,LABEL
0,0.200443,0.000121,0.043593,0.002841,0.000000,0.009122,0.039669,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.059644,0.000081,0.006703,0.002841,0.000000,0.066623,0.040868,0.000099,0.000101,0.000105,0.000094,0.000098,0.000099,0.000107,0.000106,0.000129,0.000175,0.000196,0.000278,0.000265,0.000318,0.000224,0.00023,0.000228,0.000205,0.000201,0.000213,0.000228,0.000312,0.000306,0.000303,0.000357,0.000415,0.000344,0.000368,0.000373,0.000273,0.001115,0.001119,0.001137,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.220430,0.001019,0.038322,0.000000,0.166667,0.013439,0.042426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.007205,0.000085,0.044794,0.001420,0.130952,0.010669,0.034756,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.221068,0.000000,0.043593,0.001420,0.000000,0.055872,0.040868,0.000099,0.000101,0.000105,0.000094,0.000098,0.000099,0.000107,0.000106,0.000129,0.000175,0.000196,0.000278,0.000265,0.000318,0.000224,0.00023,0.000228,0.000205,0.000201,0.000213,0.000228,0.000312,0.000306,0.000303,0.000357,0.000415,0.000344,0.000368,0.000373,0.000273,0.001115,0.001119,0.001137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [0]:
train_DF=all_DF[:59199]
val_DF=all_DF[59199:59199+6898]
test_DF=all_DF[all_DF['LABEL']==-1]

train_DF_Y=train_DF['LABEL']
val_DF_Y=val_DF['LABEL']
train_DF_normal=train_DF[train_DF['LABEL']==0]
train_DF_fire=train_DF[train_DF['LABEL']==1]

train_DF=train_DF.drop('LABEL', axis = 1)
val_DF=val_DF.drop('LABEL', axis = 1)
test_DF=test_DF.drop('LABEL', axis = 1)

train_DF_normal=train_DF_normal.drop('LABEL', axis = 1)
train_DF_fire=train_DF_fire.drop('LABEL', axis = 1)

train_DF_1=train_DF[:30000]
train_DF_Y_1=train_DF_Y[:30000]
train_DF_2=train_DF[30000:]
train_DF_Y_2=train_DF_Y[30000:]

In [0]:
col_list=train_DF.columns.to_list()

In [0]:
stack_DF=pd.DataFrame()
stack_DF_val=pd.DataFrame()
stack_DF_test=pd.DataFrame()
lda=LinearDiscriminantAnalysis()
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=5)
rf_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=42)
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

In [0]:
np.random.seed(0)
for i in range(5):
    print(i)
    rnd_list=np.random.choice(col_list, 200, replace=False)
    
    lda=lda.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['lda'+str(i)]=np.reshape(lda.transform(train_DF_2[rnd_list]),(-1,))
    stack_DF_val['lda'+str(i)]=np.reshape(lda.transform(val_DF[rnd_list]),(-1,))
    stack_DF_test['lda'+str(i)]=np.reshape(lda.transform(test_DF[rnd_list]),(-1,))
    
    decision_tree = decision_tree.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['dt'+str(i)]=np.reshape(decision_tree.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['dt'+str(i)]=np.reshape(decision_tree.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['dt'+str(i)]=np.reshape(decision_tree.predict_proba(test_DF[rnd_list])[:,1],(-1,))

    rf_clf = rf_clf.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['rf'+str(i)]=np.reshape(rf_clf.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['rf'+str(i)]=np.reshape(rf_clf.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['rf'+str(i)]=np.reshape(rf_clf.predict_proba(test_DF[rnd_list])[:,1],(-1,))

    xgb_model = xgb_model.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['xgb'+str(i)]=np.reshape(xgb_model.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['xgb'+str(i)]=np.reshape(xgb_model.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['xgb'+str(i)]=np.reshape(xgb_model.predict_proba(test_DF[rnd_list])[:,1],(-1,))

    cnn_model.fit(train_DF_1[rnd_list].as_matrix(),train_DF_Y_1.as_matrix(),epochs=50,verbose=0)
    stack_DF['cnn'+str(i)]=np.reshape(cnn_model.predict(train_DF_2[rnd_list].as_matrix()),(-1,))
    stack_DF_val['cnn'+str(i)]=np.reshape(cnn_model.predict(val_DF[rnd_list].as_matrix()),(-1,))
    stack_DF_test['cnn'+str(i)]=np.reshape(cnn_model.predict(test_DF[rnd_list].as_matrix()),(-1,))

In [27]:
stack_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
stack_model.fit(stack_DF, train_DF_Y_2)
y_pred=stack_model.predict(stack_DF_val)
print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
print ("f1_score : " , f1_score(val_DF_Y, y_pred))
print ("precision_score : " , precision_score(val_DF_Y, y_pred))
print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.7987822557262975
f1_score :  0.5088464260438783
precision_score :  0.4626769626769627
recall_score :  0.565251572327044


In [0]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(ratio='auto', kind='regular')
# X_resampled, y_resampled = sm.fit_sample(stack_DF,list(train_DF_Y_2))

In [26]:
# stack_model.fit(X_resampled, y_resampled)
# y_pred=stack_model.predict(stack_DF_val.as_matrix())
# print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
# print ("f1_score : " , f1_score(val_DF_Y, y_pred))
# print ("precision_score : " , precision_score(val_DF_Y, y_pred))
# print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.6175703102348507
f1_score :  0.4343910806174957
precision_score :  0.2986438679245283
recall_score :  0.7963836477987422


In [0]:
y_pred=stack_model.predict(stack_DF_test)
sub = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_submission.csv')
sub['fr_yn'] = y_pred
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.to_csv('/gdrive/My Drive/김해화재/PJT002_submission_hjk_stack2.csv', index=False)

In [0]:
def build_encoder():
    model=Sequential()
    model.add(Dense(16*16,activation='relu',input_shape=(436,)))
    model.add(Reshape((16,16,1)))
    model.add(Conv2D(2, 3, activation='elu', strides=2, padding='same'))
    model.add(Conv2D(4, 3, activation='elu', strides=2, padding='same'))
    model.add(Conv2D(8, 3, activation='elu', strides=2, padding='same'))
    return model

def build_decoder():
    model=Sequential()
    model.add(Conv2D(4, 3, activation='elu', padding='same'))
    model.add(UpSampling2D())
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(2, 3, activation='elu', padding='same'))
    model.add(UpSampling2D())
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(1, 3, activation='elu', padding='same'))
    model.add(UpSampling2D())
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(436,activation='sigmoid'))
    return model

In [0]:
encoder = build_encoder()
decoder = build_decoder()
AE_model=Sequential((encoder, decoder))
AE_model.compile(loss='mse',optimizer=Adam(0.001))
AE_model.fit(train_DF_fire.as_matrix(),train_DF_fire.as_matrix(),epochs=100)

In [50]:
AE_model.evaluate(train_DF_normal.as_matrix(),train_DF_normal.as_matrix())

51542/51542 [==============================] - 3s 58us/sample - loss: 0.0117


0.01173343503237919

In [0]:
fire=encoder.predict(train_DF_fire.as_matrix())
non_fire=encoder.predict(train_DF_normal.as_matrix())

In [0]:
fire=np.reshape(fire,(-1,32))
non_fire=np.reshape(non_fire,(-1,32))

In [0]:
for i in range(32):
    print(i)
    print("fire 평균",np.mean(fire[:,i]))
    print("fire 편차",np.std(fire[:,i]))
    print("non-fire 평균",np.mean(non_fire[:,i]))
    print("non-fire 편차",np.std(non_fire[:,i]))
    print()